In [13]:
#Uncomment to install
#!pip install spotipy

# Extract audio features from each song

In [14]:
from spotipy.oauth2 import SpotifyClientCredentials
import json
import spotipy
import time
import sys

from tqdm.notebook import tqdm
import pandas as pd
import os
from time import sleep

In [15]:
SPOTIPY_CLIENT_ID='a4fdae2cec0c4755bd305bc64a0aa7ea'
SPOTIPY_CLIENT_SECRET='480fac44cf5c40ef854ce3725daf7b7a'

client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace = False

CSV file created in notebook 01

In [16]:
songs_ts= pd.read_csv(os.path.join('data', 'songs_timeseries_2017_2022.csv'), index_col=0)
songs_ts.head()

/home/fterroso/.local/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,song_id,num_day,total_countries,current_countries,inc,dec
0,6m2LNopVJKsvBB9l7Z1rwn,1,1,1,1,0
1,6m2LNopVJKsvBB9l7Z1rwn,2,1,1,0,0
2,6m2LNopVJKsvBB9l7Z1rwn,3,1,1,0,0
3,6m2LNopVJKsvBB9l7Z1rwn,4,1,1,0,0
4,6m2LNopVJKsvBB9l7Z1rwn,5,1,1,0,0


In [17]:
ranking_df = pd.read_csv(os.path.join('data','clean_ranking_2017_2022.csv'), parse_dates=['Date', 'release_date'], index_col=0)
ranking_df.head()

/home/fterroso/.local/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,Position,Track Name,Artist,Streams,Date,Region,song_id,release_date
16,17,Olha a Explosão,MC Kevinho,12388,2018-01-01,Ecuador,6m2LNopVJKsvBB9l7Z1rwn,2017-01-01
19,20,Besame,"Valentino, Manuel Turizo",11127,2018-01-01,Ecuador,7Ckhk1XW5NV2k4jpqtQNlz,2017-06-03
22,23,Una Lady Como Tú,Manuel Turizo,10766,2018-01-01,Ecuador,7MHN1aCFtLXjownGhvEQlF,2017-04-12
25,26,Scooby Doo Pa Pa,Dj Kass,9879,2018-01-01,Ecuador,1xndOD8CreR5ctkOv5G1LN,2017-12-08
28,29,Déjate Llevar,"Juan Magán, Belinda, Manuel Turizo, Snova, B-Case",8995,2018-01-01,Ecuador,3xWEI23MLJrD0dpDcDUTW6,2017-11-28


In [18]:
ranking_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3600361 entries, 16 to 22869868
Data columns (total 8 columns):
 #   Column        Dtype         
---  ------        -----         
 0   Position      int64         
 1   Track Name    object        
 2   Artist        object        
 3   Streams       int64         
 4   Date          datetime64[ns]
 5   Region        object        
 6   song_id       object        
 7   release_date  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(4)
memory usage: 247.2+ MB


In [19]:
print(f'Num of songs: {songs_ts.song_id.nunique()}')
print(f'Max. total number of countries for a song: {songs_ts.total_countries.max()}')
print(f'Min. total number of countries for a song: {songs_ts.total_countries.min()}')
print(f'Max. current number of countries for a song: {songs_ts.current_countries.max()}')
print(f'Max. increment for a song: {songs_ts.inc.max()}')
print(f'Max. decrement for a song: {songs_ts.dec.max()}')

Num of songs: 8467
Max. total number of countries for a song: 68
Min. total number of countries for a song: 1
Max. current number of countries for a song: 68
Max. increment for a song: 10
Max. decrement for a song: 67


## Get songs' audio features

In [10]:
song_features_lst = []
for song_id in tqdm(songs_ts['song_id'].unique()):
    try:
        release_date = ranking_df[ranking_df['song_id']==song_id]['release_date'].values[0]
        country = ranking_df[(ranking_df['song_id']==song_id)&(ranking_df['Date']==release_date)]['Region'].values[0]

        song_features = sp.audio_features(song_id)[0]
        song_features['release_date'] = release_date
        song_features['home_country']= country
        

        #print(song_features)
        song_features_lst.append(song_features)
    except:
        print("Error with song:", song_id)

  0%|          | 0/8467 [00:00<?, ?it/s]

In [11]:
song_features_df = pd.DataFrame(song_features_lst)

In [12]:
song_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,release_date,home_country
0,0.774,0.611,6,-2.883,1,0.1340,0.1090,0.00000,0.0805,0.388,89.995,audio_features,6m2LNopVJKsvBB9l7Z1rwn,spotify:track:6m2LNopVJKsvBB9l7Z1rwn,https://api.spotify.com/v1/tracks/6m2LNopVJKsv...,https://api.spotify.com/v1/audio-analysis/6m2L...,161593,4,2017-01-01,Brazil
1,0.767,0.705,0,-6.221,1,0.0521,0.3420,0.00000,0.2500,0.520,94.013,audio_features,7Ckhk1XW5NV2k4jpqtQNlz,spotify:track:7Ckhk1XW5NV2k4jpqtQNlz,https://api.spotify.com/v1/tracks/7Ckhk1XW5NV2...,https://api.spotify.com/v1/audio-analysis/7Ckh...,232147,4,2017-06-03,Columbia
2,0.788,0.477,4,-7.348,1,0.0434,0.5490,0.00000,0.3490,0.860,94.983,audio_features,7MHN1aCFtLXjownGhvEQlF,spotify:track:7MHN1aCFtLXjownGhvEQlF,https://api.spotify.com/v1/tracks/7MHN1aCFtLXj...,https://api.spotify.com/v1/audio-analysis/7MHN...,213027,4,2017-04-12,Columbia
3,0.910,0.754,10,-2.573,0,0.2390,0.0396,0.00154,0.1220,0.630,120.939,audio_features,1xndOD8CreR5ctkOv5G1LN,spotify:track:1xndOD8CreR5ctkOv5G1LN,https://api.spotify.com/v1/tracks/1xndOD8CreR5...,https://api.spotify.com/v1/audio-analysis/1xnd...,145972,4,2017-12-08,DominicanRepublic
4,0.675,0.724,7,-5.503,1,0.0787,0.0166,0.00000,0.6970,0.506,92.051,audio_features,3xWEI23MLJrD0dpDcDUTW6,spotify:track:3xWEI23MLJrD0dpDcDUTW6,https://api.spotify.com/v1/tracks/3xWEI23MLJrD...,https://api.spotify.com/v1/audio-analysis/3xWE...,233072,4,2017-11-28,Spain


In [13]:
song_features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8467 entries, 0 to 8466
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   danceability      8467 non-null   float64       
 1   energy            8467 non-null   float64       
 2   key               8467 non-null   int64         
 3   loudness          8467 non-null   float64       
 4   mode              8467 non-null   int64         
 5   speechiness       8467 non-null   float64       
 6   acousticness      8467 non-null   float64       
 7   instrumentalness  8467 non-null   float64       
 8   liveness          8467 non-null   float64       
 9   valence           8467 non-null   float64       
 10  tempo             8467 non-null   float64       
 11  type              8467 non-null   object        
 12  id                8467 non-null   object        
 13  uri               8467 non-null   object        
 14  track_href        8467 n

In [14]:
song_features_df.to_csv(os.path.join('data', 'songs_features_2017_2022.csv'))

## Get songs' genres

In [20]:
all_genres = sp.recommendation_genre_seeds()
all_genres= all_genres['genres']
all_genres += ['rap', 'pop rock', 'trap', 'ranchera', 'corrido', 'cumbia']
all_genres

['acoustic',
 'afrobeat',
 'alt-rock',
 'alternative',
 'ambient',
 'anime',
 'black-metal',
 'bluegrass',
 'blues',
 'bossanova',
 'brazil',
 'breakbeat',
 'british',
 'cantopop',
 'chicago-house',
 'children',
 'chill',
 'classical',
 'club',
 'comedy',
 'country',
 'dance',
 'dancehall',
 'death-metal',
 'deep-house',
 'detroit-techno',
 'disco',
 'disney',
 'drum-and-bass',
 'dub',
 'dubstep',
 'edm',
 'electro',
 'electronic',
 'emo',
 'folk',
 'forro',
 'french',
 'funk',
 'garage',
 'german',
 'gospel',
 'goth',
 'grindcore',
 'groove',
 'grunge',
 'guitar',
 'happy',
 'hard-rock',
 'hardcore',
 'hardstyle',
 'heavy-metal',
 'hip-hop',
 'holidays',
 'honky-tonk',
 'house',
 'idm',
 'indian',
 'indie',
 'indie-pop',
 'industrial',
 'iranian',
 'j-dance',
 'j-idol',
 'j-pop',
 'j-rock',
 'jazz',
 'k-pop',
 'kids',
 'latin',
 'latino',
 'malay',
 'mandopop',
 'metal',
 'metal-misc',
 'metalcore',
 'minimal-techno',
 'movies',
 'mpb',
 'new-age',
 'new-release',
 'opera',
 'pagode',

In [21]:
gender_converter = {'r&b':'r-n-b', 'soundtrack':'soundtracks', 'ost':'soundtracks'}

In [22]:
song_genres_dict = {}
for song_id in tqdm(songs_ts['song_id'].unique()):
    #try:
    track_info= sp.track(song_id)
    track_genres = []
    for artist in track_info['artists']:
        artist_id=artist['id']
        artist_info= sp.artist(artist_id)
        artist_genres = ['hip-hop' if 'hip hop' in g else g for g in artist_info['genres']]
        artist_genres = [gender_converter.get(g,g) for g in artist_genres]
         
        track_genres += artist_genres #artist_info['genres']
        
        
    filtered_genres = [g for g in  track_genres if g in all_genres]
    
    if len(filtered_genres)==0:
        #print(track_genres)
        split_genres = [g.split(' ')[-1] for g in track_genres if len(g.split(' '))>1]
        split_genres += [g.split(' ')[0] for g in track_genres if len(g.split(' '))>1]
        split_genres = [gender_converter.get(g,g) for g in split_genres]
        
        filtered_genres = [g for g in split_genres if g in all_genres]

    if len(filtered_genres)==0:
        print(f'No genres from {track_genres}')
    song_genres_dict[song_id]= set(filtered_genres)
    sleep(0.01)
    #except:
    #    print("Error with song:", song_id)

  0%|          | 0/8467 [00:00<?, ?it/s]

No genres from []
No genres from ['broadway', 'hollywood', 'show tunes', 'movie tunes']
No genres from ['australian psych', 'neo-psychedelic']
No genres from ['adult standards', 'american folk revival']
No genres from ['musical advocacy']
No genres from []
No genres from ['villancicos']
No genres from ['villancicos']
No genres from ['circuit', 'guaracha', 'perreo', 'guaracha']
No genres from []
No genres from ['perreo', 'perreo']
No genres from ['adult standards', 'easy listening', 'lounge']
No genres from []
No genres from []
No genres from []
No genres from ['musica popular colombiana', 'vallenato']
No genres from ['basshall']
No genres from ['basshall']
No genres from []
No genres from ['musica popular colombiana', 'vallenato']
No genres from []
No genres from []
No genres from ['musica popular colombiana', 'vallenato']
No genres from []
No genres from ['dembow']
No genres from ['cubaton']
No genres from []
No genres from ['australian psych', 'neo-psychedelic']
No genres from ['show

No genres from ['adult standards', 'easy listening', 'lounge']
No genres from ['cartoon', "children's music"]
No genres from ['chanson']
No genres from ['eurovision', 'francoton', 'variete francaise']
No genres from ['carnaval', 'levenslied']
No genres from ['francoton']
No genres from []
No genres from []
No genres from []
No genres from []
No genres from ['chanson virale']
No genres from []
No genres from []
No genres from ['dutch drill']
No genres from ['francoton']
No genres from ['muziek voor kinderen', 'christmas product']
No genres from []
No genres from ['musical advocacy']
No genres from ['kleinkunst']
No genres from []
No genres from []
No genres from ['rkt']
No genres from ['rkt']
No genres from []
No genres from []
No genres from []
No genres from []
No genres from ['rkt']
No genres from ['urbano chileno']
No genres from []
No genres from []
No genres from ['gruperas inmortales', 'musica mexicana', 'nu-cumbia', 'tropical']
No genres from ['nueva musica mexicana']
No genres 

No genres from ['russelater']
No genres from []
No genres from []
No genres from []
No genres from ['australian reggae fusion']
No genres from ['aussie drill']
No genres from ['afroswing', 'afroswing']
No genres from ['aussie drill']
No genres from ['aussie drill']
No genres from []
No genres from ['aussie drill']
No genres from []
No genres from ['aussie drill']
No genres from []
No genres from []
No genres from []
No genres from []
No genres from ['shatta', 'dutch moombahton']
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres from []
No genres fr

No genres from ['khaleeji iraqi', 'khaliji']
No genres from ['khaleeji iraqi', 'khaliji']
No genres from ['khaleeji iraqi', 'khaliji']
No genres from ['khaleeji iraqi']
No genres from []
No genres from ['mahraganat', 'mahraganat']
No genres from ['khaleeji iraqi', 'khaliji']
No genres from ['khaleeji iraqi', 'khaliji']
No genres from ['sheilat']
No genres from ['khaliji', 'sheilat', 'khaliji', 'sheilat']
No genres from ['sheilat']
No genres from ['sheilat']
No genres from ['khaleeji iraqi']
No genres from []
No genres from ['khaleeji iraqi', 'khaliji']
No genres from ['khaleeji iraqi', 'khaliji']
No genres from ['sheilat']
No genres from []
No genres from ['sheilat']
No genres from ['khaleeji iraqi', 'khaliji']
No genres from ['sheilat']
No genres from ['sheilat']
No genres from ['khaliji', 'sheilat']
No genres from []
No genres from ['sheilat']
No genres from []
No genres from []
No genres from []
No genres from ['filmi', 'mollywood', 'mollywood']
No genres from ['mahraganat']
No genr

In [23]:
song_genres_dict

{'6m2LNopVJKsvBB9l7Z1rwn': {'funk', 'pop'},
 '7Ckhk1XW5NV2k4jpqtQNlz': {'latin', 'reggaeton'},
 '7MHN1aCFtLXjownGhvEQlF': {'latin', 'reggaeton'},
 '1xndOD8CreR5ctkOv5G1LN': {'hardcore'},
 '3xWEI23MLJrD0dpDcDUTW6': {'latin', 'pop', 'reggaeton'},
 '2ekn2ttSfGqwhhate0LSR0': {'pop'},
 '4XP6qLtHI0n9UjIPTsuqe8': {'latin', 'reggaeton'},
 '2rrJfOphzn4uQCK9X0x6R6': {'hip-hop', 'latin', 'reggaeton'},
 '5kTBmo3zNcnlm6saZRFrHR': {'latin', 'pop'},
 '05ZXXyeZFhZtA6DAP3Aw7x': {'latin', 'reggaeton'},
 '1kLTaKK1qbtGuxggUtz8iz': {'latin'},
 '6qDF4wWL49CAVbgT7yuHl8': {'latin', 'reggaeton'},
 '3IyPnNX142pvWx6D5CiimD': {'latin', 'reggaeton'},
 '5Y9fnynLlIvqtM710MHzfz': {'latin', 'reggaeton'},
 '4okba5wu9mMLXx79DXLKi3': {'hip-hop', 'latin', 'reggaeton'},
 '3WXfQDsASfXrSojk9WB3aR': {'salsa'},
 '6QJ1F9DrlsRSmKDcdVS0Oy': {'latin', 'salsa'},
 '4akm5q8AyAdbYPN2s7jxhY': {'latin', 'reggaeton'},
 '4CTjql34LKCJy5WOSgEhvA': {'hip-hop', 'latin', 'reggaeton'},
 '4Y2oUgKIeCRkN93TUzFljq': {'cumbia', 'ranchera'},
 '6QgjcU

In [24]:
song_genres_df = pd.DataFrame.from_dict(song_genres_dict, orient='index')

In [25]:
song_genres_df.head()

,0,1,2,3,4,5,6
6m2LNopVJKsvBB9l7Z1rwn,funk,pop,None,None,None,None,None
7Ckhk1XW5NV2k4jpqtQNlz,reggaeton,latin,None,None,None,None,None
7MHN1aCFtLXjownGhvEQlF,reggaeton,latin,None,None,None,None,None
1xndOD8CreR5ctkOv5G1LN,hardcore,None,None,None,None,None,None
3xWEI23MLJrD0dpDcDUTW6,reggaeton,latin,pop,None,None,None,None


In [26]:
song_genres_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8467 entries, 6m2LNopVJKsvBB9l7Z1rwn to 4eZZofkV1OQ9GDqAgHqNpA
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       7572 non-null   object
 1   1       2910 non-null   object
 2   2       1109 non-null   object
 3   3       314 non-null    object
 4   4       72 non-null     object
 5   5       10 non-null     object
 6   6       1 non-null      object
dtypes: object(7)
memory usage: 529.2+ KB


In [27]:
song_genres_df.to_csv(os.path.join('data', 'songs_genres_2017_2022.csv'))

In [28]:
print("That's all folks!")

That's all folks!
